In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import logging
from sklearn.neighbors import NearestNeighbors
import os

# Configuración de logging
logging.basicConfig(
  filename='procesamiento_datos.log',
  level=logging.INFO,
  format='%(asctime)s - %(levelname)s - %(message)s'
)

In [2]:
# Lectura de archivos Excel
try:
    estacionados_camion = pd.read_excel('../Limpia/estacionados_camion.xlsx')
    df_tareas = pd.read_excel('../Limpia/Tareas-limpio.xlsx')
    ubi_cliente = pd.read_excel('../Limpia/Ubicaciones_direcciones.xlsx')
    print("Archivos Excel leídos correctamente.")
except Exception as e:
    print(f"Error al leer los archivos Excel: {e}")
    exit()

Archivos Excel leídos correctamente.


In [3]:
print("______ INFO DF_TAREAS ______ ")
print("           ")
df_tareas.info()
print("           ")
print("______ INFO UBI_CLIENTE ______ ")
print("           ")
ubi_cliente.info()
print("           ")
print("______ INFO ESTACIONADOS_CAMION ______ ")
print("           ")
estacionados_camion.info()

______ INFO DF_TAREAS ______ 
           
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1286 entries, 0 to 1285
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   CODIGO    1286 non-null   int64 
 1   PROYECTO  1286 non-null   object
dtypes: int64(1), object(1)
memory usage: 20.2+ KB
           
______ INFO UBI_CLIENTE ______ 
           
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720 entries, 0 to 719
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   CODIGO     720 non-null    int64         
 1   NOMCLI     720 non-null    object        
 2   LATITUD    720 non-null    float64       
 3   UBICACIÓN  720 non-null    object        
 4   LONGITUD   720 non-null    float64       
 5   FECHA      720 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 33.9+ KB
           
______ INFO ESTACIO

In [4]:
estacionados_camion.tail(5)

,Indice,Numero_de_placa,Estado_de_viaje,Tiempo_de_Inicio,Tiempo_Final,Duracion,Lugar_de_inicio,camion_x,camion_y
7675,414,BYD1004,Estacionamiento,2025-02-12 15:50:26,2025-02-13 08:11:57,981.52,"34.771330S,55.757897W",-34.771330,-55.757897
7676,416,BYD1004,Estacionamiento,2025-02-13 08:14:58,2025-02-13 08:24:42,9.73,"34.768034S,55.768712W",-34.768034,-55.768712
7677,418,BYD1004,Estacionamiento,2025-02-13 08:28:18,2025-02-13 08:37:17,8.98,"34.758910S,55.772910W",-34.758910,-55.772910
7678,420,BYD1004,Estacionamiento,2025-02-13 08:46:02,2025-02-13 09:53:53,67.85,"34.778314S,55.862922W",-34.778314,-55.862922
7679,422,BYD1004,Estacionamiento,2025-02-13 09:55:20,2025-02-13 10:17:38,22.30,"34.768943S,55.862357W",-34.768943,-55.862357


## === INICIO DE ASIGNACIÓN DE CLIENTES A CAMIONES ===
### Asignación de Cliente más Cercano

In [5]:
print("\n=== INICIO DE ASIGNACIÓN DE CLIENTES A CAMIONES ===")

print("\n1. PREPARANDO DATOS PARA EL MODELO")
print("-----------------------------------")
# Usamos directamente los DataFrames originales
X = ubi_cliente[['LATITUD', 'LONGITUD']].values
y = estacionados_camion[['camion_x', 'camion_y']].values
print(f"Matriz de coordenadas de clientes (X): {X.shape}")
print(f"Matriz de coordenadas de camiones (y): {y.shape}")

if X.shape[0] == 0 or y.shape[0] == 0:
    print("❌ No hay suficientes datos con coordenadas válidas")
else:
    print("\n2. ENTRENANDO MODELO NEAREST NEIGHBORS")
    print("--------------------------------------")
    print("Configuración del modelo:")
    print("- n_neighbors: 1")
    print("- algorithm: auto")
    print("- metric: euclidean")
    print("- p: 2 (norma L2)")
    
    nbrs = NearestNeighbors(n_neighbors=1, algorithm='auto', metric='euclidean', p=2).fit(X)
    print("✓ Modelo entrenado exitosamente")

    print("\n3. CALCULANDO DISTANCIAS Y ASIGNANDO CLIENTES")
    print("--------------------------------------------")
    distances, indices = nbrs.kneighbors(y)
    
    # Estadísticas de las distancias
    print("\nEstadísticas de distancias (en grados):")
    print(f"- Distancia mínima: {distances.min():.4f}°")
    print(f"- Distancia máxima: {distances.max():.4f}°")
    print(f"- Distancia promedio: {distances.mean():.4f}°")
    print(f"- Desviación estándar: {distances.std():.4f}°")
    
    # Distribución de distancias
    print("\nDistribución de distancias:")
    percentiles = [25, 50, 75]
    for p in percentiles:
        print(f"- Percentil {p}: {np.percentile(distances, p):.4f}°")
    
    # Conteo de asignaciones
    print(f"\nTotal de asignaciones realizadas: {len(indices)}")
    
    # Análisis de clientes asignados: usamos ubi_cliente directamente
    clientes_asignados = ubi_cliente.iloc[indices.flatten()]['CODIGO'].value_counts()
    print("\nEstadísticas de asignación de clientes:")
    print(f"- Clientes únicos asignados: {len(clientes_asignados)}")
    if len(clientes_asignados) > 0:
        print(f"- Máximo de veces que se asignó un mismo cliente: {clientes_asignados.max()}")
        print("\nTop 5 clientes más asignados:")
        for codigo, count in clientes_asignados.head().items():
            print(f"  Cliente {codigo}: {count} veces")

    print("\n4. ASIGNANDO CÓDIGOS DE CLIENTES A CAMIONES")
    print("------------------------------------------")
    # Asignamos directamente sobre estacionados_camion
    estacionados_camion['CODIGO'] = ubi_cliente.iloc[indices.flatten()]['CODIGO'].values
    print("✓ Códigos asignados exitosamente")
    
    # Añadimos las distancias al DataFrame
    estacionados_camion['distancia_al_cliente'] = distances.flatten()
    print("\nEstadísticas de asignaciones:")
    print(f"- Asignaciones a menos de 1°: {(distances < 1).sum()} ({(distances < 1).sum()/len(distances)*100:.1f}%)")
    print(f"- Asignaciones a menos de 5°: {(distances < 5).sum()} ({(distances < 5).sum()/len(distances)*100:.1f}%)")
    print(f"- Asignaciones a menos de 10°: {(distances < 10).sum()} ({(distances < 10).sum()/len(distances)*100:.1f}%)")

    print("\n5. RESULTADOS FINALES")
    print("--------------------------------")
    # Como no hay filtrado, usamos directamente estacionados_camion como resultado final
    estacionados_camion_final = estacionados_camion.copy()
    total_camiones = len(estacionados_camion_final)
    camiones_asignados = estacionados_camion_final['CODIGO'].notnull().sum()
    camiones_sin_asignacion = total_camiones - camiones_asignados
    
    print(f"Total de registros en resultado final: {total_camiones}")
    print(f"- Camiones con cliente asignado: {camiones_asignados}")
    print(f"- Camiones sin asignación: {camiones_sin_asignacion}")
    
    # Resumen final de calidad de asignaciones
    print("\nResumen de calidad de asignaciones:")
    print(f"- Porcentaje de camiones con asignación: {camiones_asignados/total_camiones*100:.1f}%")
    print(f"- Distancia promedio a clientes: {distances.mean():.4f}°")
    print(f"- Mediana de distancia a clientes: {np.median(distances):.4f}°")

print("\n=== FIN DE ASIGNACIÓN DE CLIENTES A CAMIONES ===")



=== INICIO DE ASIGNACIÓN DE CLIENTES A CAMIONES ===

1. PREPARANDO DATOS PARA EL MODELO
-----------------------------------
Matriz de coordenadas de clientes (X): (720, 2)
Matriz de coordenadas de camiones (y): (7680, 2)

2. ENTRENANDO MODELO NEAREST NEIGHBORS
--------------------------------------
Configuración del modelo:
- n_neighbors: 1
- algorithm: auto
- metric: euclidean
- p: 2 (norma L2)
✓ Modelo entrenado exitosamente

3. CALCULANDO DISTANCIAS Y ASIGNANDO CLIENTES
--------------------------------------------

Estadísticas de distancias (en grados):
- Distancia mínima: 0.0000°
- Distancia máxima: 0.1673°
- Distancia promedio: 0.0008°
- Desviación estándar: 0.0032°

Distribución de distancias:
- Percentil 25: 0.0001°
- Percentil 50: 0.0003°
- Percentil 75: 0.0007°

Total de asignaciones realizadas: 7680

Estadísticas de asignación de clientes:
- Clientes únicos asignados: 492
- Máximo de veces que se asignó un mismo cliente: 552

Top 5 clientes más asignados:
  Cliente 80283: 5

## === VERIFICACION DE DF TAREAS CODIGO  ===

In [8]:
print("\n=== INICIO DE LIMPIEZA DE DATOS ===")

# Procesamiento de df_tareas
print("\n1. PROCESANDO DF_TAREAS")
print("----------------------")
print("Verificando y limpiando códigos...")

# Identificar registros inválidos
df_tareas_invalidas = df_tareas[
  df_tareas['CODIGO'].isnull() | 
  (df_tareas['CODIGO'].apply(lambda x: not str(x).isdigit()))
].copy()
print(f"Filas con 'CODIGO' nulo o no numérico: {df_tareas_invalidas.shape[0]}")

# Limpiar y convertir CODIGO
df_tareas = df_tareas.dropna(subset=['CODIGO']).copy()
df_tareas['CODIGO'] = pd.to_numeric(df_tareas['CODIGO'], errors='coerce')
filas_iniciales = df_tareas.shape[0]
df_tareas = df_tareas.dropna(subset=['CODIGO']).copy()
filas_finales = df_tareas.shape[0]
df_tareas['CODIGO'] = df_tareas['CODIGO'].astype(int)

print(f"Filas eliminadas: {filas_iniciales - filas_finales}")
print(f"Filas restantes: {filas_finales}")

# Filtrar columnas necesarias
df_tareas_filtrado = df_tareas[['CODIGO', 'PROYECTO']].copy()
print(f"Columnas seleccionadas: {list(df_tareas_filtrado.columns)}")

print("\n2. PROCESANDO ESTACIONADOS_CAMION")
print("--------------------------------")
# Convertir columnas de tiempo
for columna in ['Tiempo_de_Inicio', 'Tiempo_Final']:
  if columna in estacionados_camion.columns:
      estacionados_camion[columna] = pd.to_datetime(estacionados_camion[columna], errors='coerce')
      print(f"✓ Columna '{columna}' convertida a datetime")

# Filtrar camiones estacionados
if 'Estado_de_viaje' in estacionados_camion.columns:
  total_camiones = estacionados_camion.shape[0]
  estacionados_camion_filtered = estacionados_camion[
      estacionados_camion['Estado_de_viaje'] == 'Estacionamiento'
  ].copy()
  print(f"Camiones estacionados: {estacionados_camion_filtered.shape[0]} de {total_camiones}")
else:
  print("❌ Error: Falta columna 'Estado_de_viaje'")
  raise ValueError("Columna 'Estado_de_viaje' no encontrada")

print("\n=== FIN DE LIMPIEZA DE DATOS ===")


=== INICIO DE LIMPIEZA DE DATOS ===

1. PROCESANDO DF_TAREAS
----------------------
Verificando y limpiando códigos...
Filas con 'CODIGO' nulo o no numérico: 0
Filas eliminadas: 0
Filas restantes: 1286
Columnas seleccionadas: ['CODIGO', 'PROYECTO']

2. PROCESANDO ESTACIONADOS_CAMION
--------------------------------
✓ Columna 'Tiempo_de_Inicio' convertida a datetime
✓ Columna 'Tiempo_Final' convertida a datetime
Camiones estacionados: 7600 de 7600

=== FIN DE LIMPIEZA DE DATOS ===


In [9]:

print("\n2. PROCESANDO ESTACIONADOS_CAMION")
print("--------------------------------")
# Convertir columnas de tiempo
for columna in ['Tiempo_de_Inicio', 'Tiempo_Final']:
  if columna in estacionados_camion.columns:
      estacionados_camion[columna] = pd.to_datetime(estacionados_camion[columna], errors='coerce')
      print(f"✓ Columna '{columna}' convertida a datetime")

# Filtrar camiones estacionados
if 'Estado_de_viaje' in estacionados_camion.columns:
  total_camiones = estacionados_camion.shape[0]
  estacionados_camion_filtered = estacionados_camion[
      estacionados_camion['Estado_de_viaje'] == 'Estacionamiento'
  ].copy()
  print(f"Camiones estacionados: {estacionados_camion_filtered.shape[0]} de {total_camiones}")
else:
  print("❌ Error: Falta columna 'Estado_de_viaje'")
  raise ValueError("Columna 'Estado_de_viaje' no encontrada")

print("\n=== FIN DE LIMPIEZA DE DATOS ===")


2. PROCESANDO ESTACIONADOS_CAMION
--------------------------------
✓ Columna 'Tiempo_de_Inicio' convertida a datetime
✓ Columna 'Tiempo_Final' convertida a datetime
Camiones estacionados: 7600 de 7600

=== FIN DE LIMPIEZA DE DATOS ===


## === INICIO DE PROCESO DE MERGES ===

In [10]:
print("\n=== INICIO DE PROCESO DE MERGES ===")

print("\n1. PRIMER MERGE: df_tareas_filtrado con ubi_cliente")
print("---------------------------------------------")
print(f"Estado inicial:")
print(f"- df_tareas_filtrado: {df_tareas_filtrado.shape} filas")
print(f"- ubi_cliente: {ubi_cliente.shape} filas")

# Primer merge
merged_df = pd.merge(df_tareas_filtrado, ubi_cliente, on='CODIGO', how='inner')
print("\nResultado del primer merge:")
print(f"- Filas resultantes: {merged_df.shape[0]}")
print(f"- Columnas resultantes: {merged_df.shape[1]}")
print(f"- Columnas: {merged_df.columns.tolist()}")

print("\n2. MERGE FINAL")
print("------------")
print("Realizando merge final entre merged_df y estacionados_camion_final")
# Usamos estacionados_camion_final que ya tiene la columna CODIGO asignada
merged_final_df = pd.merge(
  merged_df, 
  estacionados_camion_final,  # Usamos el DataFrame que ya tiene CODIGO
  on='CODIGO', 
  how='inner',
  suffixes=('_tareas', '_camion')
)
print(f"\nResultado del merge final:")
print(f"- Filas: {merged_final_df.shape[0]}")
print(f"- Columnas: {merged_final_df.shape[1]}")

print("\n3. SELECCIÓN DE COLUMNAS FINALES")
print("------------------------------")
columnas_finales = [
  'CODIGO', 'PROYECTO', 'NOMCLI', 'Duracion', 'Tiempo_de_Inicio', 'UBICACIÓN',
  'Numero_de_placa', 'LATITUD', 'LONGITUD', 'camion_x', 'camion_y'
]

# Verificar columnas disponibles
columnas_faltantes = [col for col in columnas_finales if col not in merged_final_df.columns]
if columnas_faltantes:
  print("\n⚠️ Columnas faltantes:")
  for col in columnas_faltantes:
      print(f"- {col}")
else:
  filtered_df = merged_final_df[columnas_finales].copy()
  print("\nColumnas seleccionadas exitosamente:")
  for col in columnas_finales:
      print(f"- {col}")



=== INICIO DE PROCESO DE MERGES ===

1. PRIMER MERGE: df_tareas_filtrado con ubi_cliente
---------------------------------------------
Estado inicial:
- df_tareas_filtrado: (1286, 2) filas
- ubi_cliente: (720, 6) filas

Resultado del primer merge:
- Filas resultantes: 1153
- Columnas resultantes: 7
- Columnas: ['CODIGO', 'PROYECTO', 'NOMCLI', 'LATITUD', 'UBICACIÓN', 'LONGITUD', 'FECHA']

2. MERGE FINAL
------------
Realizando merge final entre merged_df y estacionados_camion_final

Resultado del merge final:
- Filas: 20883
- Columnas: 17

3. SELECCIÓN DE COLUMNAS FINALES
------------------------------

Columnas seleccionadas exitosamente:
- CODIGO
- PROYECTO
- NOMCLI
- Duracion
- Tiempo_de_Inicio
- UBICACIÓN
- Numero_de_placa
- LATITUD
- LONGITUD
- camion_x
- camion_y


## GUARDANDO RESULTADO

In [11]:

print("\n4. GUARDANDO RESULTADO")
print("--------------------")
try:
  filtered_df.to_excel('../Limpia/merged_df.xlsx', index=False)
  print("✓ Archivo guardado exitosamente")
  
  print("\nVerificación final de nulos:")
  nulos = filtered_df.isnull().sum()
  for columna, cantidad in nulos.items():
      print(f"- {columna}: {cantidad} nulos")
except Exception as e:
  print(f"❌ Error al guardar: {e}")

print("\n=== FIN DE PROCESO DE MERGES ===")


4. GUARDANDO RESULTADO
--------------------
✓ Archivo guardado exitosamente

Verificación final de nulos:
- CODIGO: 0 nulos
- PROYECTO: 0 nulos
- NOMCLI: 0 nulos
- Duracion: 0 nulos
- Tiempo_de_Inicio: 0 nulos
- UBICACIÓN: 0 nulos
- Numero_de_placa: 0 nulos
- LATITUD: 0 nulos
- LONGITUD: 0 nulos
- camion_x: 0 nulos
- camion_y: 0 nulos

=== FIN DE PROCESO DE MERGES ===


In [12]:
filtered_df.info()
filtered_df.tail(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20883 entries, 0 to 20882
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   CODIGO            20883 non-null  int32         
 1   PROYECTO          20883 non-null  object        
 2   NOMCLI            20883 non-null  object        
 3   Duracion          20883 non-null  float64       
 4   Tiempo_de_Inicio  20883 non-null  datetime64[ns]
 5   UBICACIÓN         20883 non-null  object        
 6   Numero_de_placa   20883 non-null  object        
 7   LATITUD           20883 non-null  float64       
 8   LONGITUD          20883 non-null  float64       
 9   camion_x          20883 non-null  float64       
 10  camion_y          20883 non-null  float64       
dtypes: datetime64[ns](1), float64(5), int32(1), object(4)
memory usage: 1.7+ MB


,CODIGO,PROYECTO,NOMCLI,Duracion,Tiempo_de_Inicio,UBICACIÓN,Numero_de_placa,LATITUD,LONGITUD,camion_x,camion_y
20878,12450,💲Creditos Administracion,Efeta SRL:,14.40,2025-02-01 09:13:01,"http://maps.google.com/?q=-34.78064530920805,-...",PARTNER 4251,-34.780645,-55.838592,-34.780770,-55.838409
20879,12450,💲Creditos Administracion,Efeta SRL:,23.03,2025-02-04 10:08:58,"http://maps.google.com/?q=-34.78064530920805,-...",PARTNER 4251,-34.780645,-55.838592,-34.780809,-55.838439
20880,12450,💲Creditos Administracion,Efeta SRL:,34.58,2025-02-06 09:28:50,"http://maps.google.com/?q=-34.78064530920805,-...",PARTNER 4251,-34.780645,-55.838592,-34.780865,-55.838490
20881,12450,💲Creditos Administracion,Efeta SRL:,22.95,2025-02-08 09:29:45,"http://maps.google.com/?q=-34.78064530920805,-...",PARTNER 4251,-34.780645,-55.838592,-34.780770,-55.838444
20882,12450,💲Creditos Administracion,Efeta SRL:,24.33,2025-02-11 09:44:04,"http://maps.google.com/?q=-34.78064530920805,-...",PARTNER 4251,-34.780645,-55.838592,-34.780795,-55.838382
